In [2]:
import os
from PIL import Image
import numpy as np
#--------- Code Written by Khai Tha ----------------

def adjust_bounding_boxes(bboxes, window, window_size):
    """Adjust the bounding boxes to fit within the current window."""
    adjusted_bboxes = []
    wx, wy = window
    wx_end, wy_end = wx + window_size[0], wy + window_size[1]

    for bbox in bboxes:
        # Extract the bounding box coordinates
        x_min, y_min, x_max, y_max = bbox

        # Check if the bbox is within the window
        if x_max >= wx and x_min <= wx_end and y_max >= wy and y_min <= wy_end:
            # Adjust the bounding box to the new window coordinates
            new_x_min = max(0, x_min - wx)
            new_y_min = max(0, y_min - wy)
            new_x_max = min(window_size[0], x_max - wx)
            new_y_max = min(window_size[1], y_max - wy)

            # Only include boxes that are inside the window
            if new_x_min < window_size[0] and new_y_min < window_size[1]:
                adjusted_bboxes.append([new_x_min, new_y_min, new_x_max, new_y_max])

    return adjusted_bboxes

def sliding_window_crop(image, bboxes, image_name, window_size=(640, 640), step_size=640, output_dir="output"):
    """Perform sliding window crop on the image and save the cropped images along with their adjusted bounding boxes."""
    img_width, img_height = image.size
    img_name = os.path.splitext(image_name)[0]
    counter = 0
    
    # Create output directories if they don't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    img_output_dir = os.path.join(output_dir, "images")
    lbl_output_dir = os.path.join(output_dir, "labels")
    os.makedirs(img_output_dir, exist_ok=True)
    os.makedirs(lbl_output_dir, exist_ok=True)

    # Slide the window across the image
    for y in range(0, img_height, step_size):
        for x in range(0, img_width, step_size):
            window = (x, y)
            crop = image.crop((x, y, x + window_size[0], y + window_size[1]))

            # Adjust bounding boxes to fit within the window
            adjusted_bboxes = adjust_bounding_boxes(bboxes, window, window_size)

            if adjusted_bboxes:
                # Save cropped image
                crop_name = f"{img_name}_{counter}.jpg"
                crop.save(os.path.join(img_output_dir, crop_name))

                # Save the corresponding adjusted bounding boxes
                label_name = f"{img_name}_{counter}.txt"
                with open(os.path.join(lbl_output_dir, label_name), 'w') as f:
                    for box in adjusted_bboxes:
                        x_min, y_min, x_max, y_max = box

                        # Convert absolute coordinates to normalized (relative to the crop)
                        x_center = (x_min + x_max) / 2 / window_size[0]
                        y_center = (y_min + y_max) / 2 / window_size[1]
                        width = (x_max - x_min) / window_size[0]
                        height = (y_max - y_min) / window_size[1]
                        
                        # Write normalized coordinates (YOLO format)
                        label_str = f"0 {x_center} {y_center} {width} {height}"
                        f.write(label_str + "\n")

                counter += 1

image_folder = "seconddata/cropped/images"
label_folder = "seconddata/cropped/labels"

for img_file in os.listdir(image_folder):
    if img_file.endswith(".jpg"):
        # Load the image
        img_path = os.path.join(image_folder, img_file)
        
        if not os.path.exists(img_path):
            continue
        
        image = Image.open(img_path)

        # Load the corresponding labels (assuming they're in YOLO format)
        label_path = os.path.join(label_folder, img_file.replace(".jpg", ".txt"))
        
        if not os.path.exists(label_path):
            continue
        
        bboxes = []
        with open(label_path, 'r') as f:
            for line in f.readlines():
                # Parse YOLO formatted labels (class_id, x_center, y_center, width, height)
                parts = line.strip().split()
                class_id, x_center, y_center, width, height = map(float, parts)
                img_width, img_height = image.size
                # Convert to x_min, y_min, x_max, y_max format
                x_min = int((x_center - width / 2) * img_width)
                y_min = int((y_center - height / 2) * img_height)
                x_max = int((x_center + width / 2) * img_width)
                y_max = int((y_center + height / 2) * img_height)
                bboxes.append([x_min, y_min, x_max, y_max])

        # Perform sliding window crop 
        sliding_window_crop(image, bboxes, img_file, window_size=(640, 640), step_size=640, output_dir="OGsplit/val_cropped")
